In [1]:
import sys
!{sys.executable} -m pip install PyAthena

In [2]:
import plotly
from pyathena import connect
import pandas as pd

In [3]:
q="""
select
 t1.date,
 t1.region webRegion,
 t2.city distCity,
 t1.longitude wLon,
 t1.latitude  wLat,
 t2.longitude dLon,
 t2.latitude  dLat,
 sessionCount,
 fileCount,
 pages,
 discipline
from
(
select
 date,
 g.geonetwork.region,
 g.geonetwork.longitude,
 g.geonetwork.latitude,
 h.eventinfo.eventaction webip,
 array_agg(distinct h.page.pagepath) pages,
 count(*) sessionCount
from 
 ems.google_analytics g cross join unnest(g.hits) as t(h)
where  
 g.geonetwork.country='United States'
 and h.page.hostname <> '127.0.0.1'
 and h.page.hostname <> 'localhost'
 and date = '20200412'
 and visitnumber <= 20
 and h.eventinfo is not null
 and h.eventinfo.eventcategory = 'ip address capture - master'
group by
  1,
  2,
  3,
  4,
  5
) t1,
(select
 requestdate,
 city,
 country,
 latitude,
 longitude,
 originalIP,
 array_agg(distinct discipline) discipline,
 count(*) fileCount
from 
 ems.distdata d
where
 requestDate = cast('2020-04-12' as date)
 and country = 'United States'
group by
 1,2,3,4,5,6 ) t2
where t1.webip = originalIP
order by
 t1.webip
"""
conn = connect(s3_staging_dir='s3://esdis-ems-athena', region_name='us-west-2')
pd.options.display.float_format = '{:,.0f}'.format
df = pd.read_sql(q, conn)
df

,date,webRegion,distCity,wLon,wLat,dLon,dLat,sessionCount,fileCount,pages,discipline
0,20200412,Maryland,Halethorpe,-76.6586,39.2369,-76.6907,39.2293,1,122,[oceancolor.gsfc.nasa.gov/],[OCEAN]
1,20200412,California,Davis,-121.7405,38.5449,-121.7391,38.5559,2,267,[worldview.earthdata.nasa.gov/],[ATMOSPHERE]
2,20200412,California,San Francisco,-122.4194,37.7749,-122.4952,37.7833,6,3,"[lpdaac.usgs.gov/tools/appeears/, lpdaac.usgs....","[TERRESTRIAL HYDROSPHERE, LAND SURFACE]"
3,20200412,Maryland,Hyattsville,-76.9455,38.9559,-76.9352,38.9489,1,291,[daac.ornl.gov/cgi-bin/dsviewer.pl],"[LAND SURFACE, TERRESTRIAL HYDROSPHERE, LAND S..."
4,20200412,Texas,Galveston,-94.7977,29.3013,-94.9637,29.2243,17,7,"[oceancolor.gsfc.nasa.gov/products/, oceancolo...","[LAND, BIOSPHERE, OCEAN]"
5,20200412,Texas,Barrow,-96.6389,33.1983,-156.8174,71.2346,1,1,[worldview.earthdata.nasa.gov/],[]
6,20200412,Florida,Miami,-80.2635,25.7492,-80.2702,25.7641,2,1,[oceancolor.gsfc.nasa.gov/forum/oceancolor/top...,[ATMOSPHERE]
7,20200412,Maryland,College Park,-76.9378,38.9897,-76.934,38.9965,8,16,[lpdaac.usgs.gov/data/get-started-data/collect...,[LAND]
8,20200412,South Dakota,Harrisburg,-96.7283,43.5473,-96.6864,43.446,1,5426,[lpdaac.usgs.gov/products/mod11a2v006/],[LAND SURFACE]
9,20200412,Montana,,-113.9940,46.8721,-97.822,37.751,9,49,"[wvs.earthdata.nasa.gov/, worldview.earthdata....","[, LAND]"
